#### Пример использования дедубликатора

In [1]:
%pip install faiss-cpu --no-cache

     ---------------------------------------- 10.8/10.8 MB 1.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Семплим 1000 случайных новостей

In [1]:
import pandas as pd
samples = pd.read_excel('../initial_data/posts.xlsx').sample(1000)[['title', 'text']]
samples.head()

,title,text
142301,[​](https://hooks,[​](https://hooks.pro/media/2023/07/05/bot8261...
23298,**🇩🇪 Немка пожаловалась на домогательства со с...,**🇩🇪 Немка пожаловалась на домогательства со с...
165048,"Зачем кушать сопусты, когда есть кувартошка","Зачем кушать сопусты, когда есть кувартошка"
155508,🟢купил MTLR по 186.84 RUB\ngenijefkul,🟢купил MTLR по 186.84 RUB\ngenijefkul
88181,**Скандал в **[**Ужгороде](https://t,**Скандал в **[**Ужгороде](https://t.me/+OU-Ca...


Препроцессим столбец "text" из датафрейма с семплированными новостями

In [2]:
# т.к. относительный импорт не работает необходимо подсоединить к 
# изучаемой дериктории родительскую дерикторию
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))

# препроцессим столбец с текстом
from utils import preprocessing

preprocesserClass = preprocessing.textPreprocesser(samples.copy(),['text'])
preprocesserClass.clean()

Создаем простейшие эмбеддинги на основе алгоритма TF-IDF

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = preprocesserClass.df['text'].str.join(' ')
vectorizer = TfidfVectorizer()
embeddings = vectorizer.fit_transform(corpus)
embeddings.shape

(998, 10673)

In [4]:
pd.DataFrame(embeddings.toarray(),columns=vectorizer.get_feature_names_out().tolist()) 

,aabeta,aaf,aafv,aapl,abandoned,abc,academy,activitypub,adidas,adj,...,ярк,ярославск,яростн,ярыгин,ясиноват,ясн,ят,ях,яхт,ячейк
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Находим расстояния между эмбеддингами

In [4]:
from utils import deduplicator

dists, indices = deduplicator.l2_dists(embeddings.toarray())

На основе полученных расстояний удаляем дубликаты из датафрейма с новостями

In [5]:
dedup_df, removed_n = deduplicator.dedup(preprocesserClass.df, dists, indices)
print(f'Процент удаленных дубликатов: {round(removed_n / preprocesserClass.df.shape[0] * 100, 2)}')

Процент удаленных дубликатов: 44.19


In [6]:
dedup_df.head()

,title,text
23298,**🇩🇪 Немка пожаловалась на домогательства со с...,"[немк, пожалова, на, домогательств, со, сторон..."
154293,Минувшей ночью в небе над городом,"[минувш, ноч, в, неб, над, город]"
97458,**Підведені підсумки місяця пілотного проекту ...,"[п, двед, п, дсумк, м, сят^, п, лотн, проект, ..."
100288,"Скажите, а Вы тоже, уходя из дома, говорите ко...","[скаж, а, вы, уход, из, дом, говор, кот, ил, с..."
80350,**Инвесторы вывели с начала 2023 года $27 млрд...,"[инвестор, вывел, с, нача, год, млрд, из, евро..."
